# # Homework: Week 2
Orhan Soyuhos

In [1]:
import json
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
bin_size = 0.005 #select: 0.001, 0.005, 0.01, 0.02

## -> Dataset 

In [3]:
# Python: Reading JSON

#open file
f = open('hw2.json')
hw2_data = json.load(f)
f.close()

events_data = hw2_data['events']
neurons_data = hw2_data['neurons']

event_3_data = np.array(events_data['event_3'])
event_6_data = np.array(events_data['event_6'])
sig003a_data = np.array(neurons_data['sig003a'])
sig016b_data = np.array(neurons_data['sig016b'])

events_data.keys(), neurons_data.keys()

(dict_keys(['event_3', 'event_6']), dict_keys(['sig003a', 'sig016b']))

In [4]:
len(event_3_data), len(event_6_data), len(sig003a_data), len(sig016b_data)

(77, 80, 30316, 66168)

### -> Calculate the relative_spikes

In [5]:
relative_spikes = dict()

tmp_data = list()
for event_time in event_3_data:
    tmp_data.append(list(sig003a_data - event_time))
relative_spikes['sig003a_event3'] = tmp_data

tmp_data = list()
for event_time in event_6_data:
    tmp_data.append(list(sig003a_data - event_time))
relative_spikes['sig003a_event6'] = tmp_data

tmp_data = list()
for event_time in event_3_data:
    tmp_data.append(list(sig016b_data - event_time))
relative_spikes['sig016b_event3'] = tmp_data

tmp_data = list()
for event_time in event_6_data:
    tmp_data.append(list(sig016b_data - event_time))
relative_spikes['sig016b_event6'] = tmp_data

relative_spikes.keys()

dict_keys(['sig003a_event3', 'sig003a_event6', 'sig016b_event3', 'sig016b_event6'])

### -> Calculate the relative response matrix (rrm)

In [6]:
def compute_rrm(relative_spikes, bin_size): # only for response window

    # Calculate the relative response matrix 
    rrm = []
    for relative_spike in relative_spikes:
        response_start = 0
        response_end = 0.2
        response_window = list(np.arange(response_start, response_end, bin_size))
        total_bins = len(response_window)
        # relative_spikes is the offset spike times for a given trial
        # np.histogram returns an array [histogram, bin_edges] so the call below only grabs the histogram
        binned_spikes = np.histogram(relative_spike, total_bins, range = (response_start, response_end))[0]
        rrm.append(binned_spikes)

    return rrm

In [7]:
rrm = dict() 

# sig003a_event3
relative_spi = relative_spikes['sig003a_event3']
rrm['sig003a_event3'] = compute_rrm(relative_spi, bin_size)

# sig003a_event6
relative_spi = relative_spikes['sig003a_event6']
rrm['sig003a_event6'] = compute_rrm(relative_spi, bin_size)

# sig016b_event3
relative_spi = relative_spikes['sig016b_event3']
rrm['sig016b_event3'] = compute_rrm(relative_spi, bin_size)

# sig016b_event6
relative_spi = relative_spikes['sig016b_event6']
rrm['sig016b_event6'] = compute_rrm(relative_spi, bin_size)

rrm.keys()

dict_keys(['sig003a_event3', 'sig003a_event6', 'sig016b_event3', 'sig016b_event6'])

## # Calculate the spike count probabilities

In [8]:
count_prob = dict()
H_count = dict()
rrm_sum = dict()
unique = dict()

In [9]:
# sig003a_event3
rrm_sum['sig003a_event3'] = np.sum(rrm['sig003a_event3'], 1)
unique['sig003a_event3'], frequency = np.unique(rrm_sum['sig003a_event3'], return_counts = True)
count_prob['sig003a_event3'] = frequency/sum(frequency)
H_count['sig003a_event3'] = -1 * np.sum(count_prob['sig003a_event3'] * np.log(count_prob['sig003a_event3'])/np.log(2))

# sig003a_event6
rrm_sum['sig003a_event6'] = np.sum(rrm['sig003a_event6'], 1)
unique['sig003a_event6'], frequency = np.unique(rrm_sum['sig003a_event6'], return_counts = True)
count_prob['sig003a_event6'] = frequency/sum(frequency)
H_count['sig003a_event6'] = -1 * np.sum(count_prob['sig003a_event6'] * np.log(count_prob['sig003a_event6'])/np.log(2))

# sig016b_event3
rrm_sum['sig016b_event3'] = np.sum(rrm['sig016b_event3'], 1)
unique['sig016b_event3'], frequency = np.unique(rrm_sum['sig016b_event3'], return_counts = True)
count_prob['sig016b_event3'] = frequency/sum(frequency)
H_count['sig016b_event3'] = -1 * np.sum(count_prob['sig016b_event3'] * np.log(count_prob['sig016b_event3'])/np.log(2))

# sig016b_event6
rrm_sum['sig016b_event6'] = np.sum(rrm['sig016b_event6'], 1)
unique['sig016b_event6'], frequency = np.unique(rrm_sum['sig016b_event6'], return_counts = True)
count_prob['sig016b_event6'] = frequency/sum(frequency)
H_count['sig016b_event6'] = -1 * np.sum(count_prob['sig016b_event6'] * np.log(count_prob['sig016b_event6'])/np.log(2))

In [10]:
unique_count_probab = dict()
unique_count_probab['sig003a_event3'] = np.unique(count_prob['sig003a_event3'])
unique_count_probab['sig003a_event6'] = np.unique(count_prob['sig003a_event6'])
unique_count_probab['sig016b_event3'] = np.unique(count_prob['sig016b_event3'])
unique_count_probab['sig016b_event6'] = np.unique(count_prob['sig016b_event6'])

## # Calculate the spike timing probabilities

In [11]:
timing_prob = dict()
H_timing = dict()
unique_ = dict()

In [12]:
# sig003a_event3
unique_['sig003a_event3'], frequency = np.unique(rrm['sig003a_event3'], axis = 0, return_counts = True)
timing_prob['sig003a_event3'] = frequency/sum(frequency)
H_timing['sig003a_event3'] = -1 * np.sum(timing_prob['sig003a_event3'] * np.log(timing_prob['sig003a_event3'])/np.log(2))

# sig003a_event6
unique_['sig003a_event6'], frequency = np.unique(rrm['sig003a_event6'], axis = 0, return_counts = True)
timing_prob['sig003a_event6'] = frequency/sum(frequency)
H_timing['sig003a_event6'] = -1 * np.sum(timing_prob['sig003a_event6'] * np.log(timing_prob['sig003a_event6'])/np.log(2))

# sig016b_event3
unique_['sig016b_event3'], frequency = np.unique(rrm['sig016b_event3'], axis = 0, return_counts = True)
timing_prob['sig016b_event3'] = frequency/sum(frequency)
H_timing['sig016b_event3'] = -1 * np.sum(timing_prob['sig016b_event3'] * np.log(timing_prob['sig016b_event3'])/np.log(2))

# sig016b_event6
unique_['sig016b_event6'], frequency = np.unique(rrm['sig016b_event6'], axis = 0, return_counts = True)
timing_prob['sig016b_event6'] = frequency/sum(frequency)
H_timing['sig016b_event6'] = -1 * np.sum(timing_prob['sig016b_event6'] * np.log(timing_prob['sig016b_event6'])/np.log(2))

In [13]:
unique_timing_probab = dict()
unique_timing_probab['sig003a_event3'] = np.unique(timing_prob['sig003a_event3'])
unique_timing_probab['sig003a_event6'] = np.unique(timing_prob['sig003a_event6'])
unique_timing_probab['sig016b_event3'] = np.unique(timing_prob['sig016b_event3'])
unique_timing_probab['sig016b_event6'] = np.unique(timing_prob['sig016b_event6'])

# # Mutual info for spike count
Finds the information for a neuron spike counts across stimulus

### -> Calculate p(rj|si) for spike counts
probality of response given an event

In [14]:
def check_ifInside(pair, mylist):
    inside = False
    for ii in mylist:
        isTrue = []
        for kk in range(len(ii)):
            isTrue.append(ii[kk] == pair[kk])

        if sum(isTrue) == len(ii):
            inside = True 
            break
            
    return inside

In [15]:
def prob_mutualInfo(neuron, unique, prob, case):
    
    unique_list = dict()
    tmp_count = dict()
    tmp_count[f'{neuron}_event3'] = list()
    tmp_count[f'{neuron}_event6'] = list()
    
    if case == 1:
        unique_list['for_mutualInfo'] = np.unique(np.append(unique[f'{neuron}_event3'], unique[f'{neuron}_event6']))

        idx = 0 
        kk = 0
        for ii in unique_list['for_mutualInfo']:
            if ii in unique[f'{neuron}_event3']:
                tmp_count[f'{neuron}_event3'].append(prob[f'{neuron}_event3'][kk])
                kk += 1
            else:
                tmp_count[f'{neuron}_event3'].append(0)   

        idx = 0 
        kk = 0
        for ii in unique_list['for_mutualInfo']:
            if ii in unique[f'{neuron}_event6']:
                tmp_count[f'{neuron}_event6'].append(prob[f'{neuron}_event6'][kk])
                kk += 1
            else:
                tmp_count[f'{neuron}_event6'].append(0)  
                
    elif case == 2:
        unique_list['for_mutualInfo'] = np.unique(np.concatenate([unique[f'{neuron}_event3'], unique[f'{neuron}_event6']]), axis = 0)
        
        idx = 0 
        kk = 0
        for ii in unique_list['for_mutualInfo']:
            if check_ifInside(ii, unique[f'{neuron}_event3']):
                tmp_count[f'{neuron}_event3'].append(prob[f'{neuron}_event3'][kk])
                kk += 1
            else:
                tmp_count[f'{neuron}_event3'].append(0)  

        idx = 0 
        kk = 0
        for ii in unique_list['for_mutualInfo']:
            if check_ifInside(ii, unique[f'{neuron}_event6']):
                tmp_count[f'{neuron}_event6'].append(prob[f'{neuron}_event6'][kk])
                kk += 1
            else:
                tmp_count[f'{neuron}_event6'].append(0)  

    return unique_list, tmp_count

In [16]:
# spike count related variables
sig003a = dict()
sig003a['spikeCount'] = dict()
sig003a['spikeCount'][0], sig003a['spikeCount'][1] = prob_mutualInfo('sig003a', unique, count_prob, case=1)
sig003a['spikeCount'][3] = count_prob

sig016b = dict()
sig016b['spikeCount'] = dict()
sig016b['spikeCount'][0], sig016b['spikeCount'][1] = prob_mutualInfo('sig016b', unique, count_prob, case=1)
sig016b['spikeCount'][3] = count_prob

In [17]:
# spike timing related variables
sig003a['timingCount'] = dict()
sig003a['timingCount'][0], sig003a['timingCount'][1] = prob_mutualInfo('sig003a', unique_, timing_prob, case=2)
sig003a['timingCount'][3] = timing_prob

sig016b['timingCount'] = dict()
sig016b['timingCount'][0], sig016b['timingCount'][1] = prob_mutualInfo('sig016b', unique_, timing_prob, case=2)
sig016b['timingCount'][3] = timing_prob

### -> Calculate p(rj) for spike count
probability of response across all events

In [18]:
total_trials = len(rrm_sum['sig003a_event3']) + len(rrm_sum['sig003a_event6'])
p_event = dict()
p_event['1'] = len(rrm_sum['sig003a_event3'])/total_trials
p_event['4'] = len(rrm_sum['sig003a_event6'])/total_trials

In [19]:
def calculate_p_response(neuron, neuron_dict, p_event):
    p_response = dict()
    p_response[f'{neuron}_event3'] = [x * p_event['1'] for x in neuron_dict[1][f'{neuron}_event3']]
    p_response[f'{neuron}_event6'] = [x * p_event['4'] for x in neuron_dict[1][f'{neuron}_event6']]
    p_response[neuron] = [sum(x) for x in zip(p_response[f'{neuron}_event3'], p_response[f'{neuron}_event6'])]
    
    return p_response

In [20]:
p_response = dict()
p_response['spikeCount'] = dict()
p_response['spikeCount'].update(calculate_p_response('sig003a', sig003a['spikeCount'], p_event))
p_response['spikeCount'].update(calculate_p_response('sig016b', sig016b['spikeCount'], p_event))

p_response['timingCount'] = dict()
p_response['timingCount'].update(calculate_p_response('sig003a', sig003a['timingCount'], p_event))
p_response['timingCount'].update(calculate_p_response('sig016b', sig016b['timingCount'], p_event))

### -> Calculate mutual info for spike count
Finds the information for a neuron spike counts across stimulus

In [21]:
def calculate_mutualInfo(neuron, neuron_dict, unique, p_response, p_event, case):
    
    p_r = dict()
    if case == 1: 
        p_r[f'unique_{neuron}_event3'] = [p_response[neuron][neuron_dict[0]['for_mutualInfo'].tolist().index(i)] for i in unique[f'{neuron}_event3']]
        p_r[f'unique_{neuron}_event6'] = [p_response[neuron][neuron_dict[0]['for_mutualInfo'].tolist().index(i)] for i in unique[f'{neuron}_event6']]
    elif case == 2:
        p_r[f'unique_{neuron}_event3'] = [p_response[neuron][np.flatnonzero((neuron_dict[0]['for_mutualInfo'] == i).all(1))[0]] for i in unique[f'{neuron}_event3']]
        p_r[f'unique_{neuron}_event6'] = [p_response[neuron][np.flatnonzero((neuron_dict[0]['for_mutualInfo'] == i).all(1))[0]] for i in unique[f'{neuron}_event6']]
  
    p_rj_si = dict()
    p_rj_si[f'{neuron}_event3'] = [a / b for a,b in zip(neuron_dict[3][f'{neuron}_event3'], p_r[f'unique_{neuron}_event3'])]
    p_rj_si[f'{neuron}_event6'] = [a / b for a,b in zip(neuron_dict[3][f'{neuron}_event6'], p_r[f'unique_{neuron}_event6'])]

    event_3_MI = dict()
    event_6_MI = dict()
    event_3_MI[neuron] = p_event['1'] * np.sum(neuron_dict[3][f'{neuron}_event3'] * (np.log(p_rj_si[f'{neuron}_event3'])/np.log(2)))
    event_6_MI[neuron] = p_event['4'] * np.sum(neuron_dict[3][f'{neuron}_event6'] * (np.log(p_rj_si[f'{neuron}_event6'])/np.log(2)))

    MI = event_3_MI[neuron] + event_6_MI[neuron]
        
    return MI

In [22]:
MI = dict()
MI['spikeCount'] = dict() 
MI['spikeCount']['sig003a'] = calculate_mutualInfo('sig003a', sig003a['spikeCount'], unique, p_response['spikeCount'], p_event, case=1)
MI['spikeCount']['sig016b'] = calculate_mutualInfo('sig016b', sig016b['spikeCount'], unique, p_response['spikeCount'], p_event, case=1)

In [23]:
MI['timingCount'] = dict() 
MI['timingCount']['sig003a'] = calculate_mutualInfo('sig003a', sig003a['timingCount'], unique_, p_response['timingCount'], p_event, case=2)
MI['timingCount']['sig016b'] = calculate_mutualInfo('sig016b', sig016b['timingCount'], unique_, p_response['timingCount'], p_event, case=2)

In [24]:
MI['spikeCount']

{'sig003a': 0.09944874368992068, 'sig016b': 0.1289004917394201}

In [25]:
MI['timingCount']

{'sig003a': 0.9997366009648985, 'sig016b': 0.9997366009648985}

# # Joint Mutual info for spike count
Finds the information between neurons across stimulus

### -> Calculate p(rj|si) for spike count
probability of response across neurons given specific stimulus

In [26]:
rrm_sum['event3_sig003a_sig016b'] = np.array([rrm_sum['sig003a_event3'], rrm_sum['sig016b_event3']]).T
unique['event3_sig003a_sig016b'], frequency = np.unique(rrm_sum['event3_sig003a_sig016b'], axis = 0, return_counts = True)
count_prob['event3_sig003a_sig016b'] = frequency/sum(frequency)

rrm_sum['event6_sig003a_sig016b'] = np.array([rrm_sum['sig003a_event6'], rrm_sum['sig016b_event6']]).T
unique['event6_sig003a_sig016b'], frequency = np.unique(rrm_sum['event6_sig003a_sig016b'], axis = 0, return_counts = True)
count_prob['event6_sig003a_sig016b'] = frequency/sum(frequency)

### -> Calculate p(rj|si) for spike counts
probality of response given an event

In [27]:
def check_ifInside_2(pair, mylist):
    inside = False
    for ii in mylist:
        if pair[0] == ii[0] and pair[1] == ii[1]:
            inside = True 
            break
            
    return inside

In [28]:
unique_list = dict()
unique_list['for_jointMutualInfo'] = np.unique(np.concatenate([unique['event3_sig003a_sig016b'], unique['event6_sig003a_sig016b']]), axis = 0)

tmp_count = dict()
tmp_count['event3_sig003a_sig016b'] = list()
idx = 0 
kk = 0
for ii in unique_list['for_jointMutualInfo']:
    if check_ifInside(ii, unique['event3_sig003a_sig016b']):
        tmp_count['event3_sig003a_sig016b'].append(count_prob['event3_sig003a_sig016b'][kk])
        kk += 1
    else:
        tmp_count['event3_sig003a_sig016b'].append(0)   
        
tmp_count['event6_sig003a_sig016b'] = list()
idx = 0 
kk = 0
for ii in unique_list['for_jointMutualInfo']:
    if check_ifInside(ii, unique['event6_sig003a_sig016b']):
        tmp_count['event6_sig003a_sig016b'].append(count_prob['event6_sig003a_sig016b'][kk])
        kk += 1
    else:
        tmp_count['event6_sig003a_sig016b'].append(0)  

### -> Calculate p(rj) for spike count
probability of response across all events

In [29]:
total_trials = len(rrm_sum['event3_sig003a_sig016b']) + len(rrm_sum['event6_sig003a_sig016b'])
p_event3 = len(rrm_sum['event3_sig003a_sig016b'])/total_trials
p_event6 = len(rrm_sum['event6_sig003a_sig016b'])/total_trials

In [30]:
p_response = dict()
p_response['event3_sig003a_sig016b'] = [x * p_event3 for x in tmp_count['event3_sig003a_sig016b']]
p_response['event6_sig003a_sig016b'] = [x * p_event6 for x in tmp_count['event6_sig003a_sig016b']]
p_response['sig003a_sig016b'] = [sum(x) for x in zip(p_response['event3_sig003a_sig016b'], p_response['event6_sig003a_sig016b'])]

### -> Calculate mutual info for spike count
Finds the information for a neuron spike counts across stimulus

In [31]:
p_response['unique_event3_sig003a_sig016b'] = [p_response['sig003a_sig016b'][unique_list['for_jointMutualInfo'].tolist().index(i.tolist())] for i in unique['event3_sig003a_sig016b']]
p_response['unique_event6_sig003a_sig016b'] = [p_response['sig003a_sig016b'][unique_list['for_jointMutualInfo'].tolist().index(i.tolist())] for i in unique['event6_sig003a_sig016b']]

In [32]:
p_rj_si = dict()
p_rj_si['event3_sig003a_sig016b'] = [a / b for a,b in zip(count_prob['event3_sig003a_sig016b'], p_response['unique_event3_sig003a_sig016b'])]
p_rj_si['event6_sig003a_sig016b'] = [a / b for a,b in zip(count_prob['event6_sig003a_sig016b'], p_response['unique_event6_sig003a_sig016b'])]

event_3_JMI = dict()
event_6_JMI = dict()
event_3_JMI['sig003a_sig016b'] = p_event3 * np.sum(count_prob['event3_sig003a_sig016b'] * (np.log(p_rj_si['event3_sig003a_sig016b'])/np.log(2)))
event_6_JMI['sig003a_sig016b'] = p_event6 * np.sum(count_prob['event6_sig003a_sig016b'] * (np.log(p_rj_si['event6_sig003a_sig016b'])/np.log(2)))
     
JMI = dict()    
JMI['sig003a_sig016b'] = event_3_JMI['sig003a_sig016b'] + event_6_JMI['sig003a_sig016b']
JMI['sig003a_sig016b']

0.5340078399536643

# # Synergy Redundancy

In [33]:
syn_red = JMI['sig003a_sig016b'] - MI['spikeCount']['sig003a'] - MI['spikeCount']['sig016b']  
syn_red

0.30565860452432353

# # Save to JSON file

In [34]:
hw2_results = dict()
hw2_results['event_3_prob'] = p_event3
hw2_results['event_6_prob'] = p_event6
hw2_results['joint_mutual_info'] = JMI['sig003a_sig016b'].tolist()
hw2_results['synergy_redundancy'] = syn_red.tolist()

#sig003a
hw2_results['sig003a'] = dict()
hw2_results['sig003a']['count_mutual_info'] = MI['spikeCount']['sig003a'].tolist()
hw2_results['sig003a']['timing_mutual_info'] = MI['timingCount']['sig003a'].tolist()

hw2_results['sig003a']['event_3'] = dict()
hw2_results['sig003a']['event_3']['count_entropy'] = H_count['sig003a_event3'].tolist()
hw2_results['sig003a']['event_3']['timing_entropy'] = H_timing['sig003a_event3'].tolist()
hw2_results['sig003a']['event_3']['unique_count_prob'] = unique_count_probab['sig003a_event3'].tolist()
hw2_results['sig003a']['event_3']['unique_timing_prob'] = unique_timing_probab['sig003a_event3'].tolist()

hw2_results['sig003a']['event_6'] = dict()
hw2_results['sig003a']['event_6']['count_entropy'] = H_count['sig003a_event6'].tolist()
hw2_results['sig003a']['event_6']['timing_entropy'] = H_timing['sig003a_event6'].tolist()
hw2_results['sig003a']['event_6']['unique_count_prob'] = unique_count_probab['sig003a_event6'].tolist()
hw2_results['sig003a']['event_6']['unique_timing_prob'] = unique_timing_probab['sig003a_event6'].tolist()

#sig016b
hw2_results['sig016b'] = dict()
hw2_results['sig016b']['count_mutual_info'] = MI['spikeCount']['sig016b'].tolist()
hw2_results['sig016b']['timing_mutual_info'] = MI['timingCount']['sig016b'].tolist()

hw2_results['sig016b']['event_3'] = dict()
hw2_results['sig016b']['event_3']['count_entropy'] = H_count['sig016b_event3'].tolist()
hw2_results['sig016b']['event_3']['timing_entropy'] = H_timing['sig016b_event3'].tolist()
hw2_results['sig016b']['event_3']['unique_count_prob'] = unique_count_probab['sig016b_event3'].tolist()
hw2_results['sig016b']['event_3']['unique_timing_prob'] = unique_timing_probab['sig016b_event3'].tolist()

hw2_results['sig016b']['event_6'] = dict()
hw2_results['sig016b']['event_6']['count_entropy'] = H_count['sig016b_event6'].tolist()
hw2_results['sig016b']['event_6']['timing_entropy'] = H_timing['sig016b_event6'].tolist()
hw2_results['sig016b']['event_6']['unique_count_prob'] = unique_count_probab['sig016b_event6'].tolist()
hw2_results['sig016b']['event_6']['unique_timing_prob'] = unique_timing_probab['sig016b_event6'].tolist()

In [35]:
tmp_bin = str(bin_size)[0] + str(bin_size)[2:]
tmp_bin

'0005'

In [36]:
with open(f'Soyuhos_Orhan_hw2_{tmp_bin}.json', 'w') as f_out:
    json.dump(hw2_results, f_out, indent=4, sort_keys=True)